In [86]:
# 脚本思路
# 第一步：先打开浏览器，进入网址

# 第二步：筛选条件：交易所-->产品-->合同-->日期，进入网址后就会出现所有的交易所，选择交易所后，点击查询，等待3s-4s左右，就可以看到该交易所下面所有的产品，点击产品后，等待5s左右，就可以看到对应合约。
# 选择完条件，则点击查询按钮即可查询结果。

# 第三步：将查询结果依次录入, 并导出至本地csv文档。
# 请依次运行下方第一步到第七步的代码

In [87]:
# 第一步
# 导入所需的库
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
# 根据id选择元素，返回的就是该元素对应的WebElement对象
import pandas as pd
# 需先安装上面的库，并安装webdriver

# 1.实例化Edge驱动，打开Edge浏览器
# 运行过程请勿点击Edge页面，不然会中断程序。
browser = webdriver.Edge(service = Service(r'D:\Anaconda3\Scripts\msedgedriver.exe'))
# 2.用Edge浏览器访问页面
browser.get("https://data.eastmoney.com/futures/sh/data.html")


In [88]:
# 第二步
#构建基础函数：
def data_clean(exchange_input,variety_input,contract_input,date_input,tablename):
    #开始取数
    exchange_list = []
    variety_list = []
    contract_list = []
    date_list = []
    table_type = []
    dict={}
    for i in Key_Class_id:
        x = data_extract(tablename,i)
        new_dict = {df1.loc[tablename,i]:x}
        dict.update(new_dict)
    for n in range(len(x)):
        table_type.append(tablename)
        exchange_list.append(exchange_input)
        variety_list.append(variety_input)
        contract_list.append(contract_input)
        date_list.append(date_input)
    new_dict = {
        '表名':table_type,
        '交易所':exchange_list,
        '品种':variety_list,
        '合约':contract_list,
        '日期':date_list
        }
    dict.update(new_dict)
    df_new = pd.DataFrame(dict) 
    return df_new
    

#构建抓取每列数据的函数：
def data_extract(tablename,classname):
    table_id = df1.loc[tablename,'Tab_ID']
    column = []
    element = browser.find_element(By.ID, table_id)
    e1 = element.find_elements(By.CLASS_NAME,classname)
    for i in e1:
        column.append(i.text)
    return column


# 构建日期函数：
def date_list_generator(start,end):
    date = pd.date_range(start = start, end= end).strftime("%Y-%m-%d").tolist()
    return date



In [89]:
# 第三步
# 构建数据导出函数-可自动输入参数
def data_export_as_excel_A(data,path,new_sheet_name):
    df = data
    writer = pd.ExcelWriter(path,engine="openpyxl",mode = 'a')
    # 这里mode = w为写入，a为append续写，当调整为'w'时可以用于新建excel文件，但是会覆盖原有数据，为'a'时不能用于新建，但是可以在原有excel中新增sheetname
    df.to_excel(writer,sheet_name = new_sheet_name,index = False)
    writer.save()

def data_export_as_csv_A(data,path):
    df = data
    df.to_csv(path, index=False, mode='a+', header=True,encoding='utf_8_sig')


In [77]:
# 该步骤可忽略，无需执行
# 构建数据导出函数-需要人工输入参数
def data_export_as_excel_M(data):
    path = input("请输入保存路径，如C:/Users/Lenovo/Desktop/期货龙虎榜2.xlsx:")
    new_sheet_name = input("请输入工作簿名，如GZ")
    df = data
    writer = pd.ExcelWriter(path,engine="openpyxl",mode = 'a')
    # 这里mode = w为写入，a为append续写，当调整为'w'时可以用于新建excel文件，但是会覆盖原有数据，为'a'时不能用于新建，但是可以在原有excel中新增sheetname
    df.to_excel(writer,sheet_name = new_sheet_name,index = False)
    writer.save()

def data_export_as_csv_M(data):
    path = input("请输入保存路径，如C:/Users/Lenovo/Desktop/期货龙虎榜2.csv:")
    new_sheet_name = input("请输入工作簿名，如GZ")
    df = data
    df.to_csv(path, index=False, mode='a+', header=True,encoding='utf_8_sig')
    
def data_export_M(data):
    type = input("请输入数据导出为excel还是csv格式, 输入格式为'csv':")
    data = data
    if type == 'excel':
        data_export_as_excel_M(data)
    else:
        data_export_as_csv_M(data)
    

请输入数据导出为excel还是csv格式, 输入格式为'csv':excel
请输入保存路径，如C:/Users/Lenovo/Desktop/期货龙虎榜2.xlsx:C:/Users/Lenovo/Desktop/期货龙虎榜1.xlsx
请输入工作簿名，如GZty


In [90]:
#第四步：构建基础数据
#构建表及对应id
table_name_cn = []
element = browser.find_elements(By.CLASS_NAME, 'IFtit')
for i in element[3:12]:
    table_name_cn.append(i.text)
print(table_name_cn)
table_name_id =['ulCjl','ulDtcc','ulKtcc','ulJdt','ulDtzc','ulDtjc','ulJkt','ulKtzc','ulKtjc']


# 爬取列名，构建表和对应列名
def col_name(classname):
    Col1 = browser.find_elements(By.CLASS_NAME,classname)
    Col2 = []
    for i in Col1:
        Col2.append(i.text)
    return Col2

col_name('IFe3')
#col_name('IFe4')

column_name = ['成交量', '多单量', '空单量', '净多量', '总多单', '总多单', '净空量', '总空单', '总空单']
change_name = ['增减', '增减', '增减', '增减', '增加', '减少', '增减','增加', '减少']

# 构建表名
dit1 = {
    'Tab_Name':table_name_cn,
    'Tab_ID':table_name_id,
    'V2e1':'名次',
    'V2e2':'会员简称',
    'V2e3':column_name,
    'V2e4':change_name 
}

df1 = pd.DataFrame(dit1)
df1.set_index('Tab_Name', inplace=True)
df1

Key_Class_id = ['V2e1','V2e2','V2e3','V2e4']

['成交量龙虎榜', '多头持仓龙虎榜', '空头持仓龙虎榜', '净多头龙虎榜', '多头增仓龙虎榜', '多头减仓龙虎榜', '净空头龙虎榜', '空头增仓龙虎榜', '空头减仓龙虎榜']


In [3]:
df1

,Tab_ID,V2e1,V2e2,V2e3,V2e4
Tab_Name,,,,,
成交量龙虎榜,ulCjl,名次,会员简称,成交量,增减
多头持仓龙虎榜,ulDtcc,名次,会员简称,多单量,增减
空头持仓龙虎榜,ulKtcc,名次,会员简称,空单量,增减
净多头龙虎榜,ulJdt,名次,会员简称,净多量,增减
多头增仓龙虎榜,ulDtzc,名次,会员简称,总多单,增加
多头减仓龙虎榜,ulDtjc,名次,会员简称,总多单,减少
净空头龙虎榜,ulJkt,名次,会员简称,净空量,增减
空头增仓龙虎榜,ulKtzc,名次,会员简称,总空单,增加
空头减仓龙虎榜,ulKtjc,名次,会员简称,总空单,减少


In [91]:
# 第五步：处理筛选条件
#定位筛选条件
from selenium.webdriver.support.select import Select
exchange = browser.find_element(By.ID,'futures_exchange')
variety = browser.find_element(By.ID,'futures_variety')
contract = browser.find_element(By.ID,'futures_contract')
#处理时间控件
date = browser.find_element(By.XPATH,'//*[@id="inputDate"]')
js = 'document.getElementById("inputDate").removeAttribute("readonly")'
browser.execute_script(js)

In [104]:
# 筛选条件设置说明
# 需设置data_clean(exchange_input,variety_input,contract_input,date_input,tablename)中的参数
# exchange_input:对应交易所名称，需按"广州期货交易所"格式输入
# variety_input:对应品种名称，需按"碳酸锂"格式输入
# contract_input:对应合约名称，需按"lc2407"格式输入
# date_input:对应品种名称，需按"2024-05-24"格式输入
# tablename:对应表名名称，需按"成交量龙虎榜"格式输入


# 下方核心抓数脚本为批量抓取所有数据的脚本
# 如想要修改对应参数，主要修改for xx in xx中in后面的范围:
# exchange_option即所有交易所
# variety_option即所有产品
# contract_option即所有合约
# date_sample即日期期间
# table_name_cn即要爬取的排行榜名称



In [92]:
# 第六步：本段脚本为核心抓数脚本
dict = {}
df_result = pd.DataFrame(dict)
exchange_option = browser.find_element(By.ID,'futures_exchange').find_elements(By.TAG_NAME,'option')
for i in exchange_option:
#for i in [exchange_option[-2],exchange_option[-3]]:
    # 对于每个交易所都执行下方命令
    Select(exchange).select_by_visible_text(i.text)
    variety_option = browser.find_element(By.ID,'futures_variety').find_elements(By.TAG_NAME,'option')
    for n in variety_option:
        # 对于某交易所的所有产品都执行下方命令，如"广州期货交易所"对应的所有产品
        Select(variety).select_by_visible_text(n.text)
        contract_option = browser.find_element(By.ID,'futures_contract').find_elements(By.TAG_NAME,'option')
        for m in contract_option:
            # 对于某交易所的某产品的所有合约都执行下方命令，如"广州期货交易所"对应的"碳酸锂"下对应的所有合约
            Select(contract).select_by_visible_text(m.text)
            date_sample = date_list_generator('2024-05-24','2024-05-24')
            # 请修改日期时间
            for d in date_sample:
                # 对于某交易所的某产品的某合约的所有日期都执行下方命令，如"广州期货交易所"对应的"碳酸锂"下对应的"lc2407"合约于某期间的所有数据
                date.clear()
                date.send_keys(d)
                browser.find_element(By.CLASS_NAME,'btn').click()
                time.sleep(3)
                #如果跑到中途浏览器出现问题，可以将time,sleep调成更长时间。
                for t in table_name_cn:
                    # 对于某交易所的某产品的某合约的某日期的所有排行榜都执行下方命令，如"广州期货交易所"对应的"碳酸锂"下对应的"lc2407"合约于"2024-05-24"的所有数据
                    w = data_clean(i.text,n.text,m.text,d,t)
                    df_result = pd.concat([df_result,w])

df_result

,名次,会员简称,成交量,增减,表名,交易所,品种,合约,日期,多单量,空单量,净多量,总多单,增加,减少,净空量,总空单
0,1,中信期货,4933,4536,成交量龙虎榜,广州期货交易所,工业硅,si2412,2024-05-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,中泰期货,2729,2388,成交量龙虎榜,广州期货交易所,工业硅,si2412,2024-05-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,东证期货,2577,2364,成交量龙虎榜,广州期货交易所,工业硅,si2412,2024-05-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,国富期货,1351,818,成交量龙虎榜,广州期货交易所,工业硅,si2412,2024-05-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,物产中大期货,1327,1305,成交量龙虎榜,广州期货交易所,工业硅,si2412,2024-05-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1,西部期货,NaN,NaN,多头增仓龙虎榜,郑州商品交易所,棉纱,CY2406,2024-05-24,NaN,NaN,NaN,1,1,NaN,NaN,NaN
1,2,上海大陆,NaN,NaN,多头增仓龙虎榜,郑州商品交易所,棉纱,CY2406,2024-05-24,NaN,NaN,NaN,1,1,NaN,NaN,NaN
0,1,中信建投,NaN,0,净空头龙虎榜,郑州商品交易所,棉纱,CY2406,2024-05-24,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN
1,2,广发期货,NaN,2,净空头龙虎榜,郑州商品交易所,棉纱,CY2406,2024-05-24,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN


In [93]:
# 第七步：导出数据
# 请修改path路径
path = 'C:/Users/Lenovo/Desktop/期货龙虎榜1.csv'
data_export_as_csv_A(df_result,path)
print('导出成功')

导出成功
